# Spark MLlib Statistiques de base

## Corrélation

Le calcul de la corrélation entre les deux séries de données est une opération courante dans la statistique. SparkMLlib donne la possibilité de calculer des corrélations par paires parmi de nombreuses séries. Les méthodes de corrélation prises en charge sont actuellement la corrélation de <b>Pearson et Spearman.</b>

La corrélation calcule la matrice de corrélation pour le DataSet de vecteurs en entrée à l'aide de la méthode spécifiée. La sortie sera un DataFrame contenant la matrice de corrélation de la colonne de vecteurs.

In [ ]:
import org.apache.spark.ml.linalg.{Matrix, Vectors}
import org.apache.spark.ml.stat.Correlation
import org.apache.spark.sql.Row

val data = Seq(
  Vectors.sparse(4, Seq((0, 1.0), (3, -2.0))),
  Vectors.dense(4.0, 5.0, 0.0, 3.0),
  Vectors.dense(6.0, 7.0, 0.0, 8.0),
  Vectors.sparse(4, Seq((0, 9.0), (3, 1.0)))
)

val df = data.map(Tuple1.apply).toDF("features")
val Row(coeff1: Matrix) = Correlation.corr(df, "features").head
println(s"Pearson correlation matrix:\n $coeff1")

val Row(coeff2: Matrix) = Correlation.corr(df, "features", "spearman").head
println(s"Spearman correlation matrix:\n $coeff2")

## Tests d'hypothèses

Le test d'hypothèses est un outil puissant en statistiques pour déterminer si un résultat est statistiquement significatif, si ce résultat est dû au hasard ou non. spark.ml prend actuellement en charge le chi carré de Pearson des tests d'indépendance.

In [ ]:
import org.apache.spark.ml.linalg.{Vector, Vectors}
import org.apache.spark.ml.stat.ChiSquareTest

val data = Seq(
  (0.0, Vectors.dense(0.5, 10.0)),
  (0.0, Vectors.dense(1.5, 20.0)),
  (1.0, Vectors.dense(1.5, 30.0)),
  (0.0, Vectors.dense(3.5, 30.0)),
  (0.0, Vectors.dense(3.5, 40.0)),
  (1.0, Vectors.dense(3.5, 40.0))
)

val df = data.toDF("label", "features")
val chi = ChiSquareTest.test(df, "features", "label").head
println(s"pValues = ${chi.getAs[Vector](0)}")
println(s"degreesOfFreedom ${chi.getSeq[Int](1).mkString("[", ",", "]")}")
println(s"statistics ${chi.getAs[Vector](2)}")

# statistiques synthétiques

SparkMLlib fournit des statistiques de synthèse de colonne de vecteur pour Dataframe via Summarizer. Les mesures disponibles sont les valeurs maximales, minimales, moyennes, de variance et le nombre de nonzeros dans les colonnes, ainsi que le nombre total.

In [ ]:
import org.apache.spark.ml.linalg.{Vector, Vectors}
//import org.apache.spark.mllib.stat.{MultivariateStatisticalSummary, Statistics}
import org.apache.spark.ml.stat.Summarizer

 val data = Seq(
      (Vectors.dense(2.0, 3.0, 5.0), 1.0),
      (Vectors.dense(4.0, 6.0, 7.0), 2.0)
    )

    val df = data.toDF("features", "weight")

    val (meanVal, varianceVal) = df.select(Summarizer.metrics("mean", "variance")
      .summary($"features", $"weight").as("summary"))
      .select("summary.mean", "summary.variance")
      .as[(Vector, Vector)].first()

    println(s"with weight: mean = ${meanVal}, variance = ${varianceVal}")

    val (meanVal2, varianceVal2) = df.select(Summarizer.mean($"features"), Summarizer.variance($"features"))
      .as[(Vector, Vector)].first()

    println(s"without weight: mean = ${meanVal2}, sum = ${varianceVal2}")